In [3]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
np.random.seed(1412)

In [16]:
price_quantity = np.random.randn(100000,2) + 2

In [17]:
base_price_quantity = price_quantity[0]

In [20]:
initialDF = pd.DataFrame(price_quantity,columns=["price","quantity"])
initialDF.head()

,price,quantity
0,2.336519,2.506932
1,1.901742,1.294405
2,3.111341,1.233276
3,3.079198,-0.110384
4,1.348023,0.492986


In [25]:
initialDF['cpi'] = pd.Series(initialDF["price"] * initialDF["quantity"]/base_price_quantity[0] * base_price_quantity[1])
initialDF.head()

,price,quantity,cpi
0,2.336519,2.506932,6.284707
1,1.901742,1.294405,2.641161
2,3.111341,1.233276,4.117003
3,3.079198,-0.110384,-0.364684
4,1.348023,0.492986,0.713025


In [30]:
cpi_index = pd.read_csv("data/CPIndex_Jan13-To-Jul19.csv")
cpi_index.head()

,Year,Month,State,Group,Sub Group,Description,Rural,Urban,Combined,Status,Unnamed: 10
0,2013,January,ALL India,1.0,1.1.01,Cereals and products,107.5,110.5,108.4,F,NaN
1,2013,February,ALL India,1.0,1.1.01,Cereals and products,109.2,112.9,110.4,F,NaN
2,2013,March,ALL India,1.0,1.1.01,Cereals and products,110.2,113.9,111.4,F,NaN
3,2013,April,ALL India,1.0,1.1.01,Cereals and products,110.2,114.6,111.6,F,NaN
4,2013,May,ALL India,1.0,1.1.01,Cereals and products,110.9,115.4,112.3,F,NaN


In [29]:
list(cpi_index)

['(Base 2012=100)']

In [37]:
fuel_cpi_index = cpi_index[cpi_index.Group==5.0]
fuel_cpi_index.head()

,Year,Month,State,Group,Sub Group,Description,Rural,Urban,Combined,Status,Unnamed: 10
1422,2013,January,ALL India,5.0,NaN,Fuel and light,105.5,105.4,105.5,F,NaN
1423,2013,February,ALL India,5.0,NaN,Fuel and light,106.2,105.7,106.0,F,NaN
1424,2013,March,ALL India,5.0,NaN,Fuel and light,106.1,106.0,106.1,F,NaN
1425,2013,April,ALL India,5.0,NaN,Fuel and light,106.5,106.4,106.5,F,NaN
1426,2013,May,ALL India,5.0,NaN,Fuel and light,107.5,107.2,107.4,F,NaN


In [57]:
combined_fuel_cpi = fuel_cpi_index[['Year', 'Month','Combined']]
months = {"January" : 1, "February" : 2, "March" : 3, "April" : 4, "May": 5, "June" : 6, "July" : 7, "August" : 8, "September" : 9, "October" : 10, "November" : 11, "December" : 12}
combined_fuel_cpi['Month'] = combined_fuel_cpi['Month'].map(lambda x: months[x])
combined_fuel_cpi['Timestamp'] = pd.to_datetime({'year' : combined_fuel_cpi['Year'], 'month' : combined_fuel_cpi['Month'], 'day':[1] * combined_fuel_cpi.shape[0]})
combined_fuel_cpi.head()

C:\Users\Lenovo\Anaconda3\envs\ml37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lenovo\Anaconda3\envs\ml37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Year,Month,Combined,Timestamp
1422,2013,1,105.5,2013-01-01
1423,2013,2,106.0,2013-02-01
1424,2013,3,106.1,2013-03-01
1425,2013,4,106.5,2013-04-01
1426,2013,5,107.4,2013-05-01


In [58]:
combined_fuel_cpi = combined_fuel_cpi.set_index("Timestamp")
combined_fuel_cpi.head()

,Year,Month,Combined
Timestamp,,,
2013-01-01,2013,1,105.5
2013-02-01,2013,2,106.0
2013-03-01,2013,3,106.1
2013-04-01,2013,4,106.5
2013-05-01,2013,5,107.4


In [59]:
combined_fuel_cpi = combined_fuel_cpi.drop(["Year", "Month"], axis = 1)
combined_fuel_cpi.head()

,Combined
Timestamp,
2013-01-01,105.5
2013-02-01,106.0
2013-03-01,106.1
2013-04-01,106.5
2013-05-01,107.4
